# Тематическое моделирование

Задача тематического моделирования состоит в том, чтобы разложить слова, употребляющиеся в корпусе на темы и приписать эти темы каждому документу в корпусе.

Помимо анализа тематического наполнения корпуса, мематическое моделирование может использоваться для:

1) построения вектроных представлений текстов (числа показывают насколько тема выражена в тексте). Такие представления могут быть лучше обычных (TfidfVectorizer, CountVectorizer), т.к. позволяют находить близкие документы даже если в них нет одинаковых слов (но есть слова одной тематики). Тематические векторы могут использоваться для поиска, рекомендаций или как признаки в классификации. 

2) приписывания тэгов текстам. Так можно автоматически выделять в текстовых потоках тренды, горячие темы. Тэги можно использовать как фильтр в поисковых системах. Тэги нужно приписывать темам вручную, но это проще чем размечать обучающую выборку, так как слова в тематике сразу подказывают название. 

3) составления тематических словарей



Кроме обычных текстов тематическое моделирование применяется к многим другим данным: запросам пользователей, сайтам, на которые заходят пользователи, музыке, которую слушают пользователи, покупкам в одной корзине, банковским транзакциям и даже днк. 


Все подходы к тематическому моделированию так или иначе основнованы на:

1) модели мешка слов (т.е. порядок слов в документах не учитывается)  
2) независимости документов между собой  (т.е. употребление слова W в тексте D_1 никак не влияет на слова в документе D_2)  
3) дистрибутивной гипотезе (слова, употребляющиеся вместе, объединяются в темы)



В этой тетрадке для получения тематических моделей используются LDA из gensim и NMF из sklearn.

Про LDA (и в целом тематическое моделирование) можно почитать вот эту статью - https://sysblok.ru/knowhow/kak-ponjat-o-chem-tekst-ne-chitaja-ego/

In [0]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

## Данные

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [0]:
stops = set(stopwords.words('russian')) | {'gt',}
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

# чтобы быстрее нормализовать тексты, создадим словарь всех словоформ
# нормазуем каждую 1 раз и положим в словарь
# затем пройдем по текстам и сопоставим каждой словоформе её нормальную форму

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def clear_text(text):
    fin_text = []
    for word in text:
        if is_number(word):
          fin_text.append('NUMBER')
          continue
        if len(word) == 1:
          if word.isalpha():
              fin_text.append('VARIABLE')
          continue
        if re.search('[^a-zа-я\-.0-9_]', word) is not None:
          continue
        fin_text.append(word)
    return fin_text

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        text = clear_text(text)
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [0]:
from google.colab import files

uploaded = files.upload()

Saving habr_texts.txt to habr_texts.txt


In [0]:
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

In [0]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [0]:
dictinary = gensim.corpora.Dictionary(texts)

In [0]:
 dictinary.filter_n_most_frequent(133) # ~1%

In [0]:
dictinary.filter_extremes(no_below=20)
dictinary.compactify()

In [0]:
print(dictinary)

Dictionary(7191 unique tokens: ['2-х', 'address', 'api', 'architecture', 'assembly']...)


Преобразуем наши тексты в мешки слов. 

In [0]:
corpus = [dictinary.doc2bow(text) for text in texts]

Про параметры можно почитать в документации:

In [0]:
?gensim.models.LdaMulticore

Основные это num_topics, alpha, eta и passes. 

**num_topics** - это количество тем. Это основной параметр и настраивать его проще всего. Обычно 200 оптимальное значение. Можно поставить поменьше, если тексты не очень разнообразные или хочется уменьшить время обучения.

**alpha** и **eta** - параметры, которые влияют на разреженность распределения документы-темы и темы-слова. У alpha есть значения "asymmetric" и "auto", которые можно попробовать (по умолчанию стоит "symmetric", т.е. не разреженное). Eta можно задать каким-то числом или самому сделать изначальное распределение слов по темам. НО настраивать эти параметры сложно и непонятно и вообще лучше надеяться, что по умолчанию все заработает.

**passes** - задает количество проходов по данным. Чем больше, тем лучше сойдется модель, но обучаться будет дольше.

In [0]:
counts = [150,125,100,75,50]
for count in counts:
  print(count)
  lda = gensim.models.LdaMulticore(corpus, count, id2word=dictinary, eval_every=0)
  print(lda.log_perplexity(corpus[:10000]))
  topics = []
  for topic_id, topic in lda.show_topics(num_topics=count, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)
  coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')
  print(coherence_model_lda.get_coherence())

150
-9.779373366338605
0.3825049399750696
125
-9.50526207682655
0.3588314128262319
100
-9.23573644528429
0.37737336067229293
75
-8.958502582683906
0.34460321779769243
50
-8.66988169067781
0.36856903876263347


Посмотрим на топики.

In [0]:
lda = gensim.models.LdaMulticore(corpus, 50, id2word=dictinary, eval_every=0) 

In [0]:
lda.print_topics(num_topics=50)

[(0,
  '0.007*"объект" + 0.004*"метод" + 0.004*"атрибут" + 0.003*"значение" + 0.003*"язык" + 0.003*"public" + 0.003*"вакансия" + 0.003*"класс" + 0.003*"свойство" + 0.003*"технология"'),
 (1,
  '0.005*"in" + 0.005*"метод" + 0.004*"файл" + 0.004*"элемент" + 0.004*"windows" + 0.004*"result" + 0.004*"устройство" + 0.003*"return" + 0.003*"for" + 0.003*"значение"'),
 (2,
  '0.012*"return" + 0.009*"function" + 0.009*"result" + 0.006*"файл" + 0.005*"сообщение" + 0.004*"for" + 0.004*"new" + 0.004*"let" + 0.003*"if" + 0.003*"страница"'),
 (3,
  '0.006*"сайт" + 0.005*"сервер" + 0.005*"ссылка" + 0.005*"домен" + 0.004*"файл" + 0.004*"компонент" + 0.003*"модель" + 0.003*"рынок" + 0.003*"элемент" + 0.003*"сервис"'),
 (4,
  '0.006*"return" + 0.004*"объект" + 0.004*"int" + 0.003*"технология" + 0.003*"проверка" + 0.003*"класс" + 0.003*"class" + 0.003*"атрибут" + 0.003*"студент" + 0.003*"result"'),
 (5,
  '0.013*"return" + 0.012*"result" + 0.006*"файл" + 0.006*"function" + 0.005*"for" + 0.005*"let" + 0.0

In [0]:
for i, topic in lda.print_topics(num_topics=50):
  if i == 0 or i == 20 or i == 10:
    print(i, topic)

0 0.007*"объект" + 0.004*"метод" + 0.004*"атрибут" + 0.003*"значение" + 0.003*"язык" + 0.003*"public" + 0.003*"вакансия" + 0.003*"класс" + 0.003*"свойство" + 0.003*"технология"
10 0.005*"доступ" + 0.004*"сайт" + 0.004*"метод" + 0.004*"игра" + 0.004*"устройство" + 0.004*"программа" + 0.003*"китай" + 0.003*"result" + 0.003*"пароль" + 0.003*"файл"
20 0.005*"корея" + 0.005*"игра" + 0.003*"if" + 0.003*"файл" + 0.003*"сайт" + 0.003*"язык" + 0.003*"технология" + 0.003*"result" + 0.003*"интернет" + 0.003*"инструмент"


Ещё есть штука для визуализации.

In [0]:
pyLDAvis.enable_notebook()

In [0]:
pyLDAvis.gensim.prepare(lda, corpus, dictinary) # почему-то ячейка в колабе не отрабатывает :с

In [0]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary)
corpus = tfidf[corpus]

In [0]:
counts = [150,125,100,75,50]
for count in counts:
  print(count)
  lda = gensim.models.LdaMulticore(corpus, count, id2word=dictinary, eval_every=0)
  print(lda.log_perplexity(corpus[:10000]))
  topics = []
  for topic_id, topic in lda.show_topics(num_topics=count, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)
  coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')
  print(coherence_model_lda.get_coherence())

150
-28.476810231412777
0.40361505568855194
125
-26.081525937462004
0.3901306863724088
100
-23.632931345297852
0.3890918150749223
75
-20.78084341296905
0.3778850007938987
50
-17.743723335765296
0.3654599930389017


In [0]:
lda = gensim.models.LdaMulticore(corpus, 50, id2word=dictinary, eval_every=0)
for i, topic in lda.print_topics(num_topics=50):
    print(i, topic)

0 0.012*"java" + 0.010*"репликация" + 0.009*"игра" + 0.008*"sip" + 0.006*"ботнет" + 0.005*"meetup" + 0.005*"доклад" + 0.005*"субд" + 0.005*"шлюз" + 0.005*"обратный"
1 0.030*"фсб" + 0.011*"admin" + 0.009*"russia" + 0.009*"поступить" + 0.006*"сын" + 0.004*"владимир" + 0.004*"doom" + 0.004*"take" + 0.004*"net" + 0.003*"извлечение"
2 0.021*"yahoo" + 0.020*"клетка" + 0.009*"ген" + 0.008*"age" + 0.007*"белка" + 0.006*"взлом" + 0.005*"post" + 0.004*"ткань" + 0.004*"name" + 0.004*"учётный"
3 0.017*"output" + 0.012*"less" + 0.009*"high" + 0.006*"варьироваться" + 0.006*"low" + 0.006*"фокусироваться" + 0.005*"счётчик" + 0.005*"int" + 0.004*"input" + 0.004*"усилитель"
4 0.010*"public" + 0.009*"function" + 0.008*"return" + 0.008*"class" + 0.007*"result" + 0.006*"token" + 0.005*"while" + 0.005*"файл" + 0.005*"err" + 0.004*"use"
5 0.007*"раздел" + 0.006*"аккумулятор" + 0.006*"загрузчик" + 0.006*"фоновый" + 0.005*"загадка" + 0.005*"батарея" + 0.004*"модуль" + 0.004*"загрузка" + 0.004*"сопротивление" +

У модели либо резко снижается перплексия, но немного растет когерентность, либо перплексия начинает быстро идти к 0, но и когерентность падает. Таким образом модель с использованием тфидф ведет себя хуже прошлого варианта.  
Касаемо тем:
- интересно отметить, что в первом случае первые (самые важные) слова темы повторялись среди разных тем, во втором же такого не происходит
- тема, связаная с кореей, изменилась с игр на общие слова
- пропала тема, связанная с объектами и методами (тема 0 из первого случая)



In [0]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
stexts = [' '.join(text) for text in texts]
vects = [
         TfidfVectorizer(max_features=250, min_df=10, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=500, min_df=10, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=750, min_df=10, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1000, min_df=10, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1250, min_df=10, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1500, min_df=10, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1750, min_df=10, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1000, min_df=2, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1000, min_df=5, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1000, min_df=7, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1000, min_df=12, max_df=0.3, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1000, min_df=10, max_df=0.2, ngram_range=(1,3)),
         TfidfVectorizer(max_features=1000, min_df=10, max_df=0.4, ngram_range=(1,3)),
]
for vectorizer in vects:
  X = vectorizer.fit_transform(stexts)
  model = NMF(n_components=30)
  model.fit(X)
  model.transform(X)
  print(model.reconstruction_err_)

48.97069275370468
52.44436495965947
54.129877449600066
55.13373024992506
55.74797367894692
56.30368112079559
56.669267340816084
55.14010508838048
55.10521492972835
55.09997844352421
55.12812250852275
55.822458908541506
54.717153366581634


In [0]:
vectorizer = TfidfVectorizer(max_features=250, min_df=10, max_df=0.3, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)
model = NMF(n_components=30)
model.fit(X)
model.transform(X)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [0]:
feat_names = vectorizer.get_feature_names()

In [0]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 друг  какой то  жизнь  что то
1 запрос  база  таблица  api
2 игра  игрок  мир  мобильный
3 сервер  настройка  центр  база
4 устройство  мобильный  режим  кнопка
5 the  to  of  and
6 public  void  new  string
7 язык  программирование  библиотека  обучение
8 сайт  страница  адрес  ресурс
9 сеть  интернет  связь  доступ
10 файл  строка  настройка  пакет
11 if  int  return  lt
12 модель  цена  рынок  размер
13 google  api  мобильный  рынок
14 технология  мир  материал  развитие
15 тест  тестирование  end  библиотека
16 windows  программа  компьютер  безопасность
17 элемент  блок  компонент  значение
18 виртуальный  машина  диск  управление
19 сигнал  связь  канал  скорость
20 сервис  сообщение  api  платформа
21 клиент  бизнес  интерфейс  вызов
22 объект  метод  значение  класс
23 продукт  рынок  мобильный  инструмент
24 видео  карта  изображение  экран
25 var  function  return  new
26 исследование  анализ  группа  метод
27 ключ  безопасность  открытый  доступ
28 сотрудник  бизнес  деньг

Полученные темы почти не имеют повторяющихся слов в отличие от прошлых; при этом тема про объекты и методы была получена, как и в первом случае, но не появилась тема про корею. Впрочем, полученные темы больше похоже именно на темы, а не на краткую выжимку статей, как часто получалось в прошлых случаях:
- '0.009*"модуль" + 0.008*"браузер" + 0.006*"услуга" + 0.004*"файл" + 0.004*"формат" + 0.004*"устройство" + 0.004*"игра" + 0.003*"сайт" + 0.003*"спецификация" + 0.003*"мобильный"'
- 0.018*"foreach" + 0.012*"попадание" + 0.008*"перемещение" + 0.006*"pascal" + 0.006*"корейский" + 0.005*"timestamp" + 0.005*"ru" + 0.004*"resolve" + 0.003*"посчитать" + 0.003*"семантика"  
vs.
- память  процессор  ядро  диск
- продукт  рынок  мобильный  инструмент